# Welcome to Full Stack Machine Learning's Week 4 Project!

In the final week, you will return to the workflow you built last week on the [taxi dataset](https://www.nyc.gov/site/tlc/about/tlc-trip-record-data.page). 

## Task 1: Deploy the champion
Use what you have learned in the last two weeks to make necessary modifications and to deploy your latest version of the `TaxiFarePrediction` flow to Argo. Use `--branch champion` to denote this deployment as the champion model.

In [1]:
%%writefile ../flows/cloud/taxi_fare_champion.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, trigger, project, S3
from metaflow.cards import Markdown, Table, Image, Artifact

URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'

@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name="corise_week_4")
class TaxiFarePrediction(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        # Remove noisy rows
        obviously_bad_data_filters = [
            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count > 0,
            df.total_amount > 0,
            df.trip_distance.notna(),
            df.total_amount.notna(),
        ]
        for f in obviously_bad_data_filters:
            df = df[f]
        
        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split

        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)
            
        self.df = self.transform_features(df)
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.baseline_model)

    @step
    def baseline_model(self):
        "Return mean of the data"
        from sklearn.dummy import DummyRegressor
        self.model = DummyRegressor()

        self.next(self.validate)
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Markdown(f"Score: {self.scores}"))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/taxi_fare_champion.py


In [2]:
!python ../flows/cloud/taxi_fare_champion.py --environment=conda --production --branch champion --production argo-workflows create

Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: corise_week_4, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying coriseweek4.prod.champion.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-7i4himf7igiolbsj-0-hmrb
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-7i4himf7igiolbsj-0-hmrb")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-7i4himf7igiolbsj-0-hmrb
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow coriseweek4.prod.champion.taxifareprediction for flow TaxiFarePrediction pushed t

In [3]:
!python ../flows/cloud/taxi_fare_champion.py --environment=conda --production --branch champion --production argo-workflows trigger

Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: corise_week_4, Branch: prod.champion
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow coriseweek4.prod.champion.taxifareprediction triggered on Argo Workflows (run-id argo-coriseweek4.prod.champion.taxifareprediction-xkh22).
See the run in the UI at https://ui-pw-1410177943.outerbounds.dev/TaxiFarePrediction/argo-coriseweek4.prod.champion.taxifareprediction-xkh22


## Task 2: Build the challenger
Develop a second model, by using the same `TaxiFarePrediction` architecture. Then, deploy the flow to Argo as the `--branch challenger`. 
<br>
<br>
Hint: Modify the `linear_model` step. 
<br>
Bonus: Write a paragraph summary of how you developed the second model and tested it before deploying the challenger flow. Let us know in Slack what you found challenging about the task? 

In [4]:
%%writefile ../flows/cloud/taxi_fare_challenger.py
from metaflow import FlowSpec, step, card, conda_base, current, Parameter, trigger, project, S3
from metaflow.cards import Markdown, Table, Image, Artifact

URL = 's3://outerbounds-datasets/taxi/latest.parquet'
DATETIME_FORMAT = '%Y-%m-%d %H:%M:%S'


@trigger(events=['s3'])
@conda_base(libraries={'pandas': '1.4.2', 'pyarrow': '11.0.0', 'numpy': '1.21.2', 'scikit-learn': '1.1.2'})
@project(name="corise_week_4")
class TaxiFarePrediction(FlowSpec):

    data_url = Parameter("data_url", default=URL)

    def transform_features(self, df):
        # Remove noisy rows
        obviously_bad_data_filters = [
            df.fare_amount > 0,         # fare_amount in US Dollars
            df.trip_distance <= 100,    # trip_distance in miles
            df.trip_distance > 0,
            df.passenger_count > 0,
            df.total_amount > 0,
            df.trip_distance.notna(),
            df.total_amount.notna(),
        ]
        for f in obviously_bad_data_filters:
            df = df[f]
        
        return df

    @step
    def start(self):

        import pandas as pd
        from sklearn.model_selection import train_test_split


        with S3() as s3:
            obj = s3.get(URL)
            df = pd.read_parquet(obj.path)
        
        self.df = self.transform_features(df)
        self.X = self.df["trip_distance"].values.reshape(-1, 1)
        self.y = self.df["total_amount"].values
        self.next(self.linear_model)

    @step
    def linear_model(self):
        "Fit a single variable, linear model to the data."
        from sklearn.linear_model import LinearRegression

        # TODO: Play around with the model if you are feeling it.
        self.model = LinearRegression()

        self.next(self.validate)
    
    @card(type="corise")
    @step
    def validate(self):
        from sklearn.model_selection import cross_val_score
        self.scores = cross_val_score(self.model, self.X, self.y, cv=5)
        current.card.append(Markdown("# Taxi Fare Prediction Results"))
        current.card.append(Markdown(f"Score: {self.scores}"))
        self.next(self.end)

    @step
    def end(self):
        print("Success!")


if __name__ == "__main__":
    TaxiFarePrediction()

Overwriting ../flows/cloud/taxi_fare_challenger.py


In [5]:
!python ../flows/cloud/taxi_fare_challenger.py --environment=conda --production --branch challenger --production argo-workflows create

Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: corise_week_4, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Deploying coriseweek4.prod.challenger.taxifareprediction to Argo Workflows...

The namespace of this production flow is
    production:mfprj-til6ccghzbwaugtd-0-ilds
To analyze results of this production flow add this line in your notebooks:
    namespace("production:mfprj-til6ccghzbwaugtd-0-ilds")
If you want to authorize other people to deploy new versions of this flow to Argo Workflows, they need to call
    argo-workflows create --authorize mfprj-til6ccghzbwaugtd-0-ilds
when deploying this flow to Argo Workflows for the first time.
See "Organizing Results" at https://docs.metaflow.org/ for more information about production tokens.

Bootstrapping conda environment...(this could take a few minutes)
Workflow coriseweek4.prod.challenger.taxifareprediction for flow TaxiFarePrediction pu

In [6]:
!python ../flows/cloud/taxi_fare_challenger.py --environment=conda --production --branch challenger --production argo-workflows trigger

Metaflow 2.9.2+ob(v1) executing TaxiFarePrediction for user:sandbox
Project: corise_week_4, Branch: prod.challenger
Validating your flow...
    The graph looks good!
Running pylint...
    Pylint is happy!
Workflow coriseweek4.prod.challenger.taxifareprediction triggered on Argo Workflows (run-id argo-coriseweek4.prod.challenger.taxifareprediction-wxccp).
See the run in the UI at https://ui-pw-1410177943.outerbounds.dev/TaxiFarePrediction/argo-coriseweek4.prod.challenger.taxifareprediction-wxccp


## Task 3: Analyze the results
Return to this notebook, and read in the results of the challenger and champion flow using the Metaflow Client API.
<br><br>

#### Questions
- Does your model perform better on the metrics you selected? 
- Think about your day job, how would you go about assessing whether to roll forward the production "champion" to your new model? 
    - What gives you confidence one model is better than another?
    - What kinds of information do you need to monitor to get buy-in from stakeholders that model A is preferable to model B?  

In [7]:
from metaflow import Flow, namespace

In [8]:
champion_namespace = "production:mfprj-7i4himf7igiolbsj-0-hmrb"
challenger_namespace = "production:mfprj-til6ccghzbwaugtd-0-ilds"

In [9]:
namespace(champion_namespace)
run = Flow("TaxiFarePrediction").latest_successful_run
cv_scores = run.data.scores

print("Champion - Mean score: ", sum(cv_scores) / len(cv_scores))

Champion - Mean score:  -0.0034200974812219178


In [10]:
namespace(challenger_namespace)
run = Flow("TaxiFarePrediction").latest_successful_run
cv_scores = run.data.scores

print("Champion - Mean score: ", sum(cv_scores) / len(cv_scores))

Champion - Mean score:  0.8960907470844758


## CONGRATULATIONS! 🎉✨🍾
If you made it this far, you have completed the Full Stack Machine Learning Corise course. 
We are so glad that you chose to learn with us, and hope to see you again in future courses. Stay tuned for more content and come join us in [Slack](http://slack.outerbounds.co/) to keep learning about Metaflow!